In [1]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy import spatial
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from time import time
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from hdx.data.resource import Resource
import numpy as np
import nltk
import re 
import gensim
import os;
import re 
import pandas as pd
import re;
import logging;
import sqlite3;
import time;
import sys;
import multiprocessing;
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt;
from itertools import cycle;
from io import StringIO

import pandas as pd
import requests

In [2]:
import warnings


warnings.filterwarnings('ignore')

In [3]:

import Utility

In [4]:
## Define important paramters:
Num_of_words = 20 ## 15, 20 , 30
max_DF =0.90 ## 0.85 , 0.95
k = 4 # the extra tags returned by the model.



## Keyword Extraction (TF-IDF),  Tag expansion, distance computing, and scoring similar tags for the datasets.
<ol>
<li> Read the preprocessed content </li>
<li> Generate tf-idf for the document (keywords extraction) </li>
<li> Expand tags by word2vec model </li>
<li> Represent the keyword of the dataset by word2vec model </li>
<li> Represent HDX tags by word2vec model </li>
<li> Compute the similarity between the HDX tags vs dataset </li>
</ol>



In [5]:
# read the preprocessed content:
# a list of datasets and their metadata
df=pd.read_csv("after_clean_v3.csv")



In [6]:
df.columns

Index(['title', 'notes', 'tags', 'header', 'organization', 'dataset_source',
       'geodata', 'country', 'tag_list', 'doc', 'All_text'],
      dtype='object')

In [7]:
# do cleaning for the header if it was not done
df['header'] = Utility.data_clean(df,['header'])

In [8]:
# concatenate the content of all metadata fields into raw text
def get_all_text(df):
    #df.tags.astype(str)+ " " +
    textdata =  df.title.fillna('').astype(str)+" " + df.header.fillna('').astype(str) +" " + df.organization.fillna('').astype(str) +" " +  df.notes.fillna('').astype(str)+" "+ df.country.fillna('').astype(str) + " " + df.geodata.fillna('').astype(str) 
    return textdata

In [9]:
# define the stop words list in addition standard english stopwrds
stoplist= set(stopwords.words('english'))
stoplist.update(['type','etc','also','whole','this','can','be','unnamed','nan','file','xls','xlsx','etc','zip','link',
                 'description','https','of','name'])

In [10]:
# concatenate the text of all metadata fields
df['All_text'] =  get_all_text (df)

In [11]:
# remove the stopwords 
df['text']=  Utility.remove_stopwords(df['All_text'],stoplist)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
import re

# load a set of stop words
# Get the text column
# Ignore words that appear in 85% of documents
# Eliminate stop words </li> 
docs=df['All_text'].tolist()
cv=CountVectorizer(max_df=0.85,stop_words=stoplist,binary=True)#, ngram_range =(1,2))
word_count_vector=cv.fit_transform(docs)
print(word_count_vector)

  (0, 2131)	1
  (0, 620)	1
  (0, 1058)	1
  (0, 743)	1
  (0, 2312)	1
  (0, 1167)	1
  (0, 1000)	1
  (0, 2363)	1
  (0, 1592)	1
  (0, 1066)	1
  (0, 385)	1
  (0, 256)	1
  (0, 2392)	1
  (0, 2110)	1
  (0, 1540)	1
  (0, 2372)	1
  (0, 985)	1
  (0, 1183)	1
  (0, 2233)	1
  (0, 2253)	1
  (0, 1705)	1
  (0, 388)	1
  (0, 2142)	1
  (0, 1385)	1
  (0, 1369)	1
  :	:
  (611, 2048)	1
  (611, 511)	1
  (611, 2344)	1
  (611, 2069)	1
  (611, 924)	1
  (611, 2050)	1
  (611, 521)	1
  (611, 224)	1
  (611, 2231)	1
  (611, 1512)	1
  (611, 1097)	1
  (611, 853)	1
  (611, 621)	1
  (611, 425)	1
  (611, 410)	1
  (611, 354)	1
  (611, 1516)	1
  (611, 865)	1
  (611, 343)	1
  (611, 2065)	1
  (611, 2156)	1
  (611, 1517)	1
  (611, 1744)	1
  (611, 1059)	1
  (611, 692)	1


In [13]:
# term frequncy inverse document frequncy transformer  
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=False)
tfidf_transformer.fit(word_count_vector)
 

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=False)

In [14]:
TfidfTransformer.get_params

<function sklearn.base.BaseEstimator.get_params(self, deep=True)>

In [15]:
##quick look at top words
list(cv.vocabulary_.keys())[:3]

['somalia', 'demographic', 'health']

In [16]:
# a mapping of index to word name
feature_names=cv.get_feature_names()

In [17]:
## use this as a test to see the top words in the docuemnt
# get the document that we want to extract keywords from
doc=df['All_text'].ix[100]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc])) #tfidf_transformer
 
#sort the tf-idf vectors by descending order of scores
sorted_items=Utility.sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 15
keywords=Utility.extract_topn_from_vector(feature_names,sorted_items,15)
 
# now print the results
print("\n=====Doc=====")
#print(doc)
print(" current tags " , df['tags'].ix[100])

print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
 current tags  demographics education indicators socioeconomics sustainable development

===Keywords===
world 0.11
women 0.11
wider 0.11
venezuela 0.11
various 0.11
unesco 0.11
today 0.11
ties 0.11
sustainable 0.11
strengthens 0.11
specialized 0.11
socioeconomic 0.11
societies 0.11
serves 0.11
serve 0.11


In [18]:
# get the document that we want to extract keywords from.
#generate tf-idf for the given document.
# sort the tf-idf vectors by descending order of scores.
#extract only the top n; n here is 10

tfidf_vecs = []
df['tfidf'] =" "

for i, row in df.iterrows():
    doc=df['All_text'].ix[i]
    candidates=""
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc])) #tfidf_transformer.transform(
    sorted_items=Utility.sort_coo(tf_idf_vector.tocoo()) 
    keywords=Utility.extract_topn_from_vector(feature_names,sorted_items,Num_of_words)
    vec =np.array(list(keywords.values()))
    for k in keywords: 
        candidates+=k +" "
    tfidf_vecs.append(vec)
    df.ix[i]['tfidf'] = candidates  
 

In [19]:
df['text'].ix[21]

'ghana education indicators uneca ghana education indicators uneca united nations economic commission africa created title united nations economic commission africa uneca organization true state active image url revision id fb c e ce e c b aa c organization aa ae cf c c e approval status approved dataset contains many indicators education enrolment rate primary education ratio girls boys primary education list find ghana'

In [20]:
df['tfidf'].ix[21]

'united uneca ratio rate primary nations many list indicators girls ghana find fb enrolment education economic dataset contains commission cf '

In [21]:
# Google pretrained word2vec model:
model, word = Utility.get_model()
index2word_set = set(model.wv.index2word)

In [22]:
## read the list of HDX valid tags
tag_list_clean = Utility.read_concept_tags()
len(tag_list_clean)

369

In [23]:
## do tag expansion using word2vec simialr words
new_tag_list_clean = []
for tag in tag_list_clean:
    expanded = Utility.expan_tags(tag, model , 5)
    if expanded =="":
        new_tag_list_clean.append(tag)
    else:
        new_tag_list_clean.append(expanded+" "+tag)
    

In [44]:
#create testing data set

evaluation_set = df.sample(100)

print(evaluation_set)

#evaluation_set = pd.read_csv('good_test.csv')

                                                 title  \
578                   secondary education school rates   
22                   gambia education indicators uneca   
199  liechtenstein sustainable development educatio...   
423  jordan demographic health education transport ...   
327              cabo verde education indicators uneca   
..                                                 ...   
512                uganda number girls enrolled school   
126  south sudan sustainable development education ...   
413  malawi demographic health education transport ...   
4                              hotosm dominica schools   
356  turkmenistan demographic health education tran...   

                                                 notes  \
578  secondary education school rates including bre...   
22   this dataset contains many indicators educatio...   
199  contains data from unesco data portal covering...   
423  urban indicators data available here analyzed ...   
327  this dat

In [45]:
## example of cosine distance
Utility.cosine_distance(df['tfidf'].ix[20], tag_list_clean,model,index2word_set,10)

[('water points - water sources', 0.5745146),
 ('water sanitation and hygiene - wash', 0.53523254),
 ('water bodies - hydrography', 0.5220669),
 ('people targeted for assistance', 0.51016665),
 ('access to water', 0.5070385),
 ('hazardous areas', 0.5036735),
 ('social development centers', 0.50205225),
 ('host communities', 0.49804816),
 ('people in need - pin', 0.49558908),
 ('cultural sites', 0.4949108)]

In [46]:
'''
This code snippets- for each dataset:
 1) get the document that we want to extract keywords from
 2) generate tf-idf for the document
 3) sort the tf-idf vectors by descending order of scores
 4) extract only the top ; n here is 20 (Num_of_words)
 5) clean the content (number, special character,links and/or non-english chars removal )
 6) concatenate the text content from all metdata fields.
'''
evaluation_set['tfidf'] =""
for i, row in evaluation_set.iterrows():
    doc=evaluation_set.ix[i]['All_text']
    candidates=""
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc])) #tfidf_transformer.transform(
    sorted_items=Utility.sort_coo(tf_idf_vector.tocoo()) 
    keywords=Utility.extract_topn_from_vector(feature_names,sorted_items,Num_of_words)
    for k in keywords:
        candidates+=k +" "
    evaluation_set.at[i,'tfidf'] = candidates      

In [52]:
## example of cosine distance
Utility.cosine_distance(evaluation_set['tfidf'].iloc[0], tag_list_clean,model,index2word_set,10)

[('affected population', 0.5433383),
 ('people targeted for assistance', 0.50946397),
 ('access to education', 0.49615312),
 ('vulnerable populations', 0.49022922),
 ('population movement', 0.4862483),
 ('education facilities - schools', 0.48593074),
 ('persons of concern - populations of concern - poc', 0.48053625),
 ('people in need - pin', 0.47038212),
 ('mine risk education', 0.4692416),
 ('social development centers', 0.4653482)]

In [35]:
## convert the string of list (tags) to list of (tags) 
#problem for reading list from CSV
from ast import literal_eval
evaluation_set.loc[:,'perfect_tags'] = evaluation_set.loc[:,'perfect_tags'].apply(lambda x: literal_eval(x))

KeyError: 'perfect_tags'

In [29]:
## example of cosine distance
Utility.cosine_distance(evaluation_set['tfidf'].ix[98], tag_list_clean,model,index2word_set,10)

[('governance and civil society', 0.52650523),
 ('social development centers', 0.52534616),
 ('sustainable development', 0.5186519),
 ('educational attainment', 0.49680942),
 ('education cluster', 0.46917826),
 ('health services - healthcare', 0.4673657),
 ('sustainable development goals - sdg', 0.46731296),
 ('reproductive health and family planning', 0.4597844),
 ('host communities', 0.45850664),
 ('religious facilities', 0.45708698)]

In [30]:
# Actual list: is the list of tags currenly assigned to the dataset by the providers.
# pred_list: The list of tags suggested by the model.
#tfidf_list: list of keywords extracted by tfidf
# tf_listL list of keywords extracted by term frequncy
# perfect_list: list of valid tags.
actual_flist= evaluation_set['tag_list']
pred_list= evaluation_set['rec']
tfidf_list = evaluation_set['tfidf']
tf_list = evaluation_set['tf']
perfect_list = evaluation_set['perfect_tags']

KeyError: 'rec'

In [ ]:
# compare top words with the perfect list:
# an idea to overcome the exact matching.
result_tfidf=[]
for document_lst, pred_tags in zip(perfect_list,tfidf_list):
    matching= [s for s in document_lst if any(s in xs or xs in s for xs in [pred_tags])]
    result_tfidf.append((len(matching), len(document_lst)))
        

In [ ]:
### compuate the recall for the keywords. 

tfidf_recall = []
for t in result_tfidf:
    tfidf_recall.append(t[0]/t[1])
np.average(tfidf_recall)

In [ ]:
## Optional - tag cleaning
P_tags =  Utility.clean_tag(new_tag_list_clean)
good_tags = Utility.remove_stopwords(pd.Series(P_tags), stoplist)

In [ ]:
## example of cosine distance
Utility.cosine_distance("secondary data secondary nigeria data sdr review sdr review nigeria education education secondary data review sdr secondary sdr nigeria matrix nigeria matrix education nigeria education education data matrix", tag_list_clean,model,index2word_set,10)

In [ ]:
# run the model on the evaluation set 
# represent the tfidf of the documents into word2vec. 
# Find top tags from the clean_list to each dataset using the 
df['rec'] = ""
for i, row in df.iterrows():
    doc = df.ix[i]['tfidf']
    #if i ==12:
     #   print(doc)
    suggested_tags =Utility.cosine_distance_list(doc, tag_list_clean,model,index2word_set,5 )#evaluation_set.ix[i]['k']+5) 
    #print(suggested_tags)
    df.at[i, 'rec'] = suggested_tags

In [ ]:
##final list
## represent the tfidf of the documents into word2vec
##recommend tags from the hdx valid tag list to each dataset based on cosine distance 
for i, row in evaluation_set.iterrows():
    doc = evaluation_set.ix[i]['tfidf']
    suggested_tags =Utility.cosine_distance_list(doc, good_tags,model,index2word_set,evaluation_set.ix[i]['k']+5)
    #print(suggested_tags)
    evaluation_set.at[i, 'rec'] = suggested_tags

In [ ]:
pred_list= evaluation_set['rec']

In [ ]:
####compare word2vec of top words with the perfect list:
## an idea for the comparsion
res_word2vec=[]
total = 0
for document_lst, pred_tags in zip(perfect_list,pred_list):
    matching= [s for s in document_lst if any(s in xs or xs in s for xs in pred_tags)]
    print("document_lst", document_lst)
    print("predicted", pred_list)
    res_word2vec.append((len(matching), len(document_lst)))
        

In [ ]:
word2vec_recall = []
for t in res_word2vec:
    word2vec_recall.append(t[0]/t[1])
print(np.mean(word2vec_recall))